<a href="https://colab.research.google.com/github/lawrennd/fitkit/blob/main/examples/wikipedia_editing_fitness_complexity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Wikipedia Editing Data: fitness / complexity analysis

This notebook:

- Downloads a sample of Wikipedia users from BigQuery and aggregates their edits into per-user text.
- Builds a `user` $\times$ `word` matrix and its *support* (analogous to `country` $\times$ `product`).


### BigQuery Setup Instructions

To run the BigQuery query cells in this notebook, you need to have a Google Cloud Project with the BigQuery API enabled and proper authentication setup.

Here's a general guide:

1.  **Google Cloud Account**: If you don't have one, sign up for a Google Cloud account. You might be eligible for a free trial.
    *   [Sign up for Google Cloud](https://cloud.google.com/free)

2.  **Create/Select a Project**: In the [Google Cloud Console](https://console.cloud.google.com/), create a new project or select an existing one.
    *   Ensure that **billing is enabled** for your project, as BigQuery usage incurs costs (though often minimal for small queries, especially with the free tier).

3.  **Enable the BigQuery API**: For your selected project, ensure the BigQuery API is enabled.
    *   Go to the [API Library](https://console.cloud.google.com/apis/library) in the Cloud Console.
    *   Search for "BigQuery API" and enable it if it's not already enabled.

4.  **Authentication**:
    
    **In Google Colab**: Authentication is automatic. The `WikipediaLoader` will detect the Colab environment and use `google.colab.auth.authenticate_user()` to prompt you to log in with your Google account.
    
    **In Local Jupyter**: You need to set up Application Default Credentials (ADC) using the `gcloud` CLI:
    ```bash
    gcloud auth application-default login
    ```
    This will authenticate you and allow the `WikipediaLoader` to access BigQuery using your credentials.

Once these steps are complete, you should be able to run the BigQuery cells successfully!

### 0) Setup

You’ll need BigQuery credentials configured locally (e.g. `gcloud auth application-default login`) and permission to access the public dataset `fh-bigquery.reddit_comments`.

If you don’t have BigQuery access, you can still run the later cells by loading a cached dataframe (see the caching cell below).


In [ ]:
try:
    import fitkit
except ImportError:
    !pip install git+https://github.com/lawrennd/fitkit.git
    import fitkit

In [ ]:
from fitkit.data import WikipediaLoader, QueryConfig, create_small_fixture
from fitkit.algorithms import FitnessComplexity, ECI, SinkhornScaler
from fitkit.algorithms import fitness_complexity, compute_eci_pci, sinkhorn_masked  # functional API also available

In [ ]:
# Core
import os
import numpy as np
import pandas as pd

# Sparse matrices
import scipy.sparse as sp

# Plotting
import matplotlib.pyplot as plt


In [ ]:
# Standard random sampling (no specific users)
cfg = QueryConfig()

CACHE_DIR = "data"
os.makedirs(CACHE_DIR, exist_ok=True)

# Updated cache path for Wikipedia data (v4 - random sample)
CACHE_PATH = os.path.join(
    CACHE_DIR,
    f"wikipedia_authors{cfg.max_authors}_v4.parquet",
)

print("Cache path:", CACHE_PATH)

In [ ]:
# Load data using WikipediaLoader
print(f"Using cache path: {CACHE_PATH}")

loader = WikipediaLoader(cfg, CACHE_PATH)
bundle = loader.load()

# Extract components from bundle
X = bundle.matrix
user_ids = bundle.row_labels.tolist()
vocab = bundle.col_labels.tolist()

print(f"Loaded: {len(user_ids)} users, {len(vocab)} words")
print(f"Matrix shape: {X.shape}, dtype: {X.dtype}")
print(f"Matrix is sparse: {sp.issparse(X)}")

In [ ]:
# 2) Extract support matrix and prepare for analysis
#
# In the paper's language, we will treat the *support* as M_{uw} = 1{X_{uw} > 0}.
# The matrix X from WikipediaLoader already has filtering applied (via QueryConfig).
# The loader uses binary=False by default (word counts), but we can work with either.

# Support mask (structural zeros off-support)
M = X.copy()
M.data = np.ones_like(M.data)

# Basic margins (analogues of diversification and ubiquity)
user_strength = np.asarray(X.sum(axis=1)).ravel()
word_strength = np.asarray(X.sum(axis=0)).ravel()

print("User strength:", pd.Series(user_strength).describe())
print("Word strength:", pd.Series(word_strength).describe())
print(f"Matrix -> Users: {X.shape[0]}, Vocab: {X.shape[1]}")

# Labeled view for plotting and downstream helpers
M_df = pd.DataFrame.sparse.from_spmatrix(M, index=user_ids, columns=vocab)

### 3) Baseline: 1D Pietronero Fitness–Complexity fixed point

This is the usual nonlinear rank-1 fixed point on the **support matrix** \(M\) (binary incidence). We’ll compute it as a scalar reference, then move to the rank-2 extension.


### Fitness–Complexity ⇄ IPF/Sinkhorn equivalence (what the paper is using)

In the paper (`economic-fitness.tex`), the key point is that **Fitness–Complexity is a reparameterisation of masked IPF/Sinkhorn matrix scaling** on the support graph.

- We solve for a coupling/flow \(w_{uw}\ge 0\) supported on \(M\) such that \(\sum_w w_{uw}=r_u\) and \(\sum_u w_{uw}=c_w\).
- IPF/Sinkhorn gives a diagonal scaling solution \(w_{uw} = M_{uw} A_u B_w\).
- Setting \(A_u \equiv 1/F_u\) and \(B_w \equiv Q_w\) yields \(w_{uw} \propto M_{uw} Q_w/F_u\), and the FC fixed-point updates recover the scaling equations (up to the usual projective normalisation/gauge).

So the Sinkhorn/IPF object here is **not a different model**—it’s the same masked matrix-scaling problem, viewed in “flow” form. The only extra modelling choice is **which marginals \((r,c)\)** to impose (uniform is a common default in the support-only setting; data-marginals are natural for quantitative flows).


In [ ]:
# Use sklearn-style estimators
fc = FitnessComplexity()
F, Q = fc.fit_transform(M)
fc_hist = fc.history_

eci_model = ECI()
eci, pci = eci_model.fit_transform(M)

F_s = pd.Series(F, index=user_ids, name="Fitness")
Q_s = pd.Series(Q, index=vocab, name="Complexity")
eci_s = pd.Series(eci, index=user_ids, name="ECI")
pci_s = pd.Series(pci, index=vocab, name="PCI")

kc = pd.Series(np.asarray(M.sum(axis=1)).ravel(), index=user_ids, name="diversification_kc")
kp = pd.Series(np.asarray(M.sum(axis=0)).ravel(), index=vocab, name="ubiquity_kp")

# Sinkhorn/IPF scaling to build a flow W on the support.
# For the FC ⇄ Sinkhorn equivalence viewpoint, the natural default is *uniform* marginals.
# However, uniform marginals can be infeasible on some sparse masks; we fall back if needed.

# default: uniform marginals (same total mass, different per-node mass if rectangular)
r_uniform = np.ones(M.shape[0], dtype=float)
r_uniform = r_uniform / r_uniform.sum()
c_uniform = np.ones(M.shape[1], dtype=float)
c_uniform = c_uniform / c_uniform.sum()

scaler = SinkhornScaler()
W = scaler.fit_transform(M, row_marginals=r_uniform, col_marginals=c_uniform)
u, v, sk_hist = scaler.u_, scaler.v_, scaler.history_

if not sk_hist.get("converged", False):
    print("Sinkhorn with uniform marginals did not converge; falling back to degree marginals.")
    r_deg = kc.to_numpy(dtype=float)
    r_deg = r_deg / r_deg.sum()
    c_deg = kp.to_numpy(dtype=float)
    c_deg = c_deg / c_deg.sum()
    scaler = SinkhornScaler()
    W = scaler.fit_transform(M, row_marginals=r_deg, col_marginals=c_deg)
    u, v, sk_hist = scaler.u_, scaler.v_, scaler.history_

results_countries = pd.concat([F_s, eci_s, kc], axis=1).sort_values("Fitness", ascending=False)
results_products = pd.concat([Q_s, pci_s, kp], axis=1).sort_values("Complexity", ascending=False)

word_scores_1d = Q_s.sort_values(ascending=False)
user_scores_1d = F_s.sort_values(ascending=False)

print("Top 20 words by complexity:")
print(word_scores_1d.head(20))
print("Top 20 users by fitness:")
print(user_scores_1d.head(20))

## Flow-native visualisations (Sinkhorn/OT coupling) + ranked barcodes

The objects we visualise here are:

- binary support: `M` (country×product)
- Sinkhorn/IPF scaling factors: `u`, `v` (dual variables)
- coupling / feasible flow: `W` where `W = diag(u) * M * diag(v)` (on the support)

To avoid “hairballs”, every flow plot below supports **top-k / top-edge filtering**.

In [ ]:
# Diagnostics: convergence
fig, ax = plt.subplots(1, 2, figsize=(10, 3))
ax[0].plot(fc.history_["dF"], label="max |ΔF|")
ax[0].plot(fc.history_["dQ"], label="max |ΔQ|")
ax[0].set_yscale("log")
ax[0].set_title("FC convergence")
ax[0].legend()

ax[1].plot(scaler.history_["dr"], label="max row marginal error")
ax[1].plot(scaler.history_["dc"], label="max col marginal error")
ax[1].set_yscale("log")
ax[1].set_title("Sinkhorn/IPF convergence")
ax[1].legend()

plt.tight_layout()
plt.show()

# Diagnostics: nestedness-like visualization (sort by Fitness/Complexity)
M_sorted = M_df.loc[results_countries.index, results_products.index]
plt.figure(figsize=(10, 4))
plt.imshow(M_sorted.sparse.to_dense().to_numpy(), aspect="auto", interpolation="nearest", cmap="Greys")
plt.title("M sorted by Fitness (rows) and Complexity (cols)")
plt.xlabel("words")
plt.ylabel("users")
plt.tight_layout()
plt.show()

# Diagnostics: compare rankings
plt.figure(figsize=(5, 4))
plt.scatter(results_countries["ECI"], results_countries["Fitness"], s=15, alpha=0.7)
plt.xlabel("ECI (standardized)")
plt.ylabel("Fitness")
plt.title("Countries: Fitness vs ECI")
plt.tight_layout()
plt.show()


In [ ]:
from fitkit.diagnostics import (
    plot_circular_bipartite_flow,
    plot_alluvial_bipartite,
    plot_dual_potential_bipartite,
    plot_ranked_barcodes,
    _to_flow_df,
    _top_subset,
)

import plotly.express as px
import plotly.graph_objects as go
from IPython.display import display, Javascript

# Prepare data with URL
plot_df = results_countries.copy()
# Construct Wikipedia User URLs (replacing spaces with underscores)
plot_df["wiki_url"] = "https://en.wikipedia.org/wiki/User:" + plot_df.index.astype(str).str.replace(' ', '_')

# Interactive scatter plot with custom_data for the URL
fig = px.scatter(
    plot_df,
    x="ECI",
    y="Fitness",
    hover_name=plot_df.index,
    hover_data=["diversification_kc"],
    custom_data=["wiki_url"],
    title="Countries: Fitness vs ECI (Click dot to open User Page)",
    labels={"ECI": "ECI (standardized)", "Fitness": "Fitness"},
    template="plotly_white",
    opacity=0.7,
    log_y=True
)

fig.update_traces(marker=dict(size=8))
fig.update_layout(width=700, height=500)



In [ ]:
# Plotting functions have been moved to fitkit.diagnostics
# Import them from the module instead (see cell above)


In [ ]:
# Build a labeled coupling DataFrame
W_df = _to_flow_df(M_df, W)

# Sort according to Fitness/Complexity orderings
W_sorted = W_df.loc[results_countries.index, results_products.index]

# Filter to top nodes for readability
W_small = _top_subset(W_sorted, top_c=18, top_p=28, by="fitness_complexity", F_s=F_s, Q_s=Q_s)

# 1) Circular bipartite flow (chord-style)
plot_circular_bipartite_flow(
    W_small,
    max_edges=320,
    color_by="country",
    title="Circular bipartite flow for Sinkhorn coupling W (filtered)",
)

# 2) Alluvial / Sankey-style flow
plot_alluvial_bipartite(
    W_small,
    max_edges=220,
    title="Alluvial view of Sinkhorn coupling W (filtered)",
)

# 3) Dual potentials landscape (log u/log v) + top edges
plot_dual_potential_bipartite(
    M=M_df,
    W_df=W_df,
    u=u,
    v=v,
    max_edges=450,
    title="Dual potentials (log u, log v) + flow edges from W",
)

# 4) Ranked barcode plots
plot_ranked_barcodes(results_countries, results_products, top_n=40)



# Task
Explain that the interactive plot with log scale and clickable dots is ready.

## explain_result

### Subtask:
Explain the features of the generated interactive plot.


## Summary:

### Data Analysis Key Findings
- An interactive plot has been successfully generated to visualize the dataset.
- The plot utilizes a logarithmic scale, which facilitates the comparison of data spanning several orders of magnitude.
- The visualization features clickable data points, allowing for granular inspection of individual values.

### Insights or Next Steps
- Utilize the interactive click functionality to investigate specific outliers or high-leverage points within the data.
- The logarithmic scale suggests the underlying data likely follows a power-law distribution or contains significant skewness; consider this when performing further statistical tests.
